## Imports

In [ ]:
import findspark
findspark.init('/home/SalmaEssam/spark-3.5.3-bin-hadoop3')




from pyspark.sql.functions import max
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, format_number
from pyspark.sql.functions import mean, stddev
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import year, month
from pyspark.sql.functions import dayofyear , dayofmonth
from pyspark.sql.functions import min, max

## Spark Session

In [3]:
spark = SparkSession.builder.appName('Project').getOrCreate()

24/12/24 14:57:47 WARN Utils: Your hostname, mohamedsalah-VirtualBox resolves to a loopback address: 127.0.1.1; using 192.168.1.5 instead (on interface enp0s3)
24/12/24 14:57:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/24 14:57:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Read a dataset

In [4]:
df = spark.read.csv('BTC-USD.csv', inferSchema = True, header = True)


[Stage 0:>                                                          (0 + 1) / 1]



## 1. What are the columns names?

In [5]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']

## 2. What does the Schema look like?

In [6]:
df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: long (nullable = true)
 |-- Dividends: integer (nullable = true)
 |-- Stock Splits: integer (nullable = true)



## 3. Display the first 15 rows

In [7]:
df.show(15)

+----------+----------------+----------------+----------------+----------------+-----------+---------+------------+
|      Date|            Open|            High|             Low|           Close|     Volume|Dividends|Stock Splits|
+----------+----------------+----------------+----------------+----------------+-----------+---------+------------+
|2019-05-31| 8320.2861328125| 8586.6591796875|   8172.55078125|  8574.501953125|25365190957|        0|           0|
|2019-06-01|   8573.83984375| 8625.6005859375|     8481.578125| 8564.0166015625|22488303544|        0|           0|
|2019-06-02| 8565.4736328125| 8809.3037109375| 8561.2353515625| 8742.9580078125|20266216022|        0|           0|
|2019-06-03| 8741.7470703125|          8743.5|  8204.185546875| 8208.9951171875|22004511436|        0|           0|
|2019-06-04| 8210.9853515625| 8210.9853515625|7564.48876953125|7707.77099609375|24609731549|        0|           0|
|2019-06-05|7704.34326171875|7901.84912109375|7668.66845703125| 7824.231

## 4. Describe the dataset in statistical measurements

In [8]:
df.describe().show()

24/12/24 14:58:04 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+------------------+------------------+------------------+--------------------+---------+------------+
|summary|              Open|              High|               Low|             Close|              Volume|Dividends|Stock Splits|
+-------+------------------+------------------+------------------+------------------+--------------------+---------+------------+
|  count|               759|               759|               759|               759|                 759|      759|         759|
|   mean|18881.256430644764|19422.984229609272|18271.106332731186| 18911.30140462883|3.559935378705006...|      0.0|         0.0|
| stddev| 16221.57494286444|16755.792555279113|15566.983081696353|16222.805201441279|2.213068308485810...|      0.0|         0.0|
|    min|       5002.578125|    5331.833984375|  4106.98095703125|   4970.7880859375|         11445355859|        0|           0|
|    max|    63523.75390625|    64863.09765625|    62208.96484375|    63503.45703125|     

## 5. Reshape the results of the previous question into a suitable format

In [9]:

df.describe().select(
    col("summary"),
    format_number(col("Open").cast("float"), 2).alias("Open"),
    format_number(col("High").cast("float"), 2).alias("High"),
    format_number(col("Low").cast("float"), 2).alias("Low"),
    format_number(col("Close").cast("float"), 2).alias("Close"),
    format_number(col("Volume").cast("float"), 2).alias("Volume"),
    format_number(col("Dividends").cast("float"), 2).alias("Dividends"),
    format_number(col("Stock Splits").cast("float"), 2).alias("Stock Splits")
).show()

+-------+---------+---------+---------+---------+------------------+---------+------------+
|summary|     Open|     High|      Low|    Close|            Volume|Dividends|Stock Splits|
+-------+---------+---------+---------+---------+------------------+---------+------------+
|  count|   759.00|   759.00|   759.00|   759.00|            759.00|   759.00|      759.00|
|   mean|18,881.26|19,422.98|18,271.11|18,911.30| 35,599,351,808.00|     0.00|        0.00|
| stddev|16,221.58|16,755.79|15,566.98|16,222.81| 22,130,683,904.00|     0.00|        0.00|
|    min| 5,002.58| 5,331.83| 4,106.98| 4,970.79| 11,445,355,520.00|     0.00|        0.00|
|    max|63,523.75|64,863.10|62,208.96|63,503.46|350,967,955,456.00|     0.00|        0.00|
+-------+---------+---------+---------+---------+------------------+---------+------------+



## 6. Add a new detail that displays the ratio of the Low Price versus Close

In [10]:
df = df.withColumn("Low_to_Close_Ratio", col("Low") / col("Close"))

df.show(15)

+----------+----------------+----------------+----------------+----------------+-----------+---------+------------+------------------+
|      Date|            Open|            High|             Low|           Close|     Volume|Dividends|Stock Splits|Low_to_Close_Ratio|
+----------+----------------+----------------+----------------+----------------+-----------+---------+------------+------------------+
|2019-05-31| 8320.2861328125| 8586.6591796875|   8172.55078125|  8574.501953125|25365190957|        0|           0|0.9531225050653225|
|2019-06-01|   8573.83984375| 8625.6005859375|     8481.578125| 8564.0166015625|22488303544|        0|           0|0.9903738537186559|
|2019-06-02| 8565.4736328125| 8809.3037109375| 8561.2353515625| 8742.9580078125|20266216022|        0|           0|0.9792149686539021|
|2019-06-03| 8741.7470703125|          8743.5|  8204.185546875| 8208.9951171875|22004511436|        0|           0|0.9994141097364732|
|2019-06-04| 8210.9853515625| 8210.9853515625|7564.4887

## 7. What day had the peak volume? 

In [11]:
peak_volume_row = df.orderBy(col("Volume").desc()).first()

peak_date = peak_volume_row["Date"]
peak_volume = peak_volume_row["Volume"]

print(f"The day with the peak volume is {peak_date} with a volume of {peak_volume}.")

The day with the peak volume is 2021-02-26 with a volume of 350967941479.


## 8. What is the max of the Volume column? 

In [14]:
df.select((max("Volume")).alias('The maximum Volume')).show()

+------------------+
|The maximum Volume|
+------------------+
|      350967941479|
+------------------+



## 9. What is the average and the standard deviation of the opening? 

In [15]:
df.select(mean("Open").alias('Average of the Open column'),
          stddev("Open").alias('Standard deviation of the Open column')).show()

+--------------------------+-------------------------------------+
|Average of the Open column|Standard deviation of the Open column|
+--------------------------+-------------------------------------+
|        18881.256430644764|                    16221.57494286444|
+--------------------------+-------------------------------------+



## 10. How many days was the opening lower than 10,000 dollars? 

In [16]:
days_below_10000 = df.filter(col("Open") < 10000).count()
days_below_10000

337

## 11. What percentage of the time was the closing greater than 32 dollars?

In [17]:
total_days = df.count()

days_above_32 = df.filter(col("Close") > 32).count()

percentage_above_32 = (days_above_32 / total_days) * 100

print(f"The percentage of days where the closing price was greater than $32 is {percentage_above_32:.2f}%.")

The percentage of days where the closing price was greater than $32 is 100.00%.


## 12. How many distinct volumes?

In [18]:
distinct_volumes = df.select("Volume").distinct().count()
distinct_volumes

759

#### another sol.

In [19]:
df.select(countDistinct('Volume').alias('Number of distinct volumes')).show()

+--------------------------+
|Number of distinct volumes|
+--------------------------+
|                       759|
+--------------------------+



## 13. What is the min opening per year?

In [20]:
min_open_per_year = (
    df.withColumn("Year", year(col("Date")))  
      .groupBy("Year")                        
      .agg(min("Open").alias("Min_Open"))     
      .orderBy("Year")                        
)

min_open_per_year.show()

+----+---------------+
|Year|       Min_Open|
+----+---------------+
|2019|6647.6982421875|
|2020|    5002.578125|
|2021|28994.009765625|
+----+---------------+



## 14. What is the highest closing for each day of the year?

In [21]:
df = df.withColumn("Day",dayofyear("Date"))

HighestDays = df.select("Day","Close").groupBy("Day").max()
HighestDays.select("Day","max(Close)").orderBy('Day').show(366)

+---+----------------+
|Day|      max(Close)|
+---+----------------+
|  1|  29374.15234375|
|  2| 32127.267578125|
|  3|   32782.0234375|
|  4|   31971.9140625|
|  5|   33992.4296875|
|  6|  36824.36328125|
|  7|  39371.04296875|
|  8|    40797.609375|
|  9|    40254.546875|
| 10|  38356.44140625|
| 11|     35566.65625|
| 12|   33922.9609375|
| 13|    37316.359375|
| 14|    39187.328125|
| 15|   36825.3671875|
| 16|    36178.140625|
| 17|  35791.27734375|
| 18|  36630.07421875|
| 19|   36069.8046875|
| 20|        35547.75|
| 21|  30825.69921875|
| 22|  33005.76171875|
| 23| 32067.642578125|
| 24|  32289.37890625|
| 25| 32366.392578125|
| 26| 32569.849609375|
| 27|    30432.546875|
| 28|  31649.60546875|
| 29|  34316.38671875|
| 30|   34269.5234375|
| 31|    33114.359375|
| 32|  33537.17578125|
| 33|   35510.2890625|
| 34|  37472.08984375|
| 35|  36926.06640625|
| 36|  38144.30859375|
| 37|  39266.01171875|
| 38|  38903.44140625|
| 39|  46196.46484375|
| 40|  46481.10546875|
| 41|  4491